In [ ]:
# default_exp classify_ions

In [ ]:
#export
import numpy as np
import pandas as pd
import scipy.stats
def collect_node_parameters(all_nodes, w_annot = True):
    ion2param2val = {}
    all_headers = set()
    for node in all_nodes:

        param2val = get_param2val(node)
        if w_annot:
            param2val.update({"ion" : node.name})
        if node.type == 'mod_seq_charge':
            if len(node.children) ==2:
                fcfc_diff = abs(node.children[0].fc - node.children[1].fc)
                param2val.update({"ms1_ms2_fcfc_diff" : fcfc_diff})

            for child in node.children:
                name_frac_mainclust = f"child_type_{child.type}_frac_mainclust"
                name_num_mainclusts = f"child_type_{child.type}_num_mainclusts"
                name_variance = f"child_type_{child.type}_cv_fcs"
                name_replicate_cv = f"child_type_{child.type}_replicate_cv"
                param2val.update({name_frac_mainclust: child.frac_mainclust, name_num_mainclusts : child.num_mainclusts, name_variance : calc_variance_for_node(child), name_replicate_cv : child.cv})
                clusterstats_dict = calc_cluster_stats(child)
                param2val.update(clusterstats_dict)
        all_headers.update(param2val.keys())
        ion2param2val[node.name] = param2val

    return get_dataframe(all_nodes,ion2param2val, all_headers)


def get_param2val(node):
    headers_of_interest = ["frac_mainclust", "num_mainclusts", "fraction_consistent",  "cv", "min_intensity", "min_reps"]
    param_dict = node.__dict__
    param2val = {x: param_dict.get(x) for x in headers_of_interest}
    param2val["num_leaves"] = len(node.leaves)
    if len(node.children)>0:
        param2val["num_children"] = len(node.children)
        param2val["cv_fcs"] = calc_variance_for_node(node)
    return param2val

def get_dataframe(all_nodes,ion2param2val, all_headers):
    all_headers = list(all_headers)
    rows = [[ion2param2val.get(node.name).get(header, np.nan) for header in all_headers] for node in all_nodes]
    df = pd.DataFrame(rows, columns = all_headers)
    return df


def calc_cluster_stats(node):
    num_elems_secondclust = 0
    fcs_clust0 = []
    fcs_clust1 = []
    intensities = []
    for child in node.children:
        if child.cluster ==0:
            fcs_clust0.append(child.fc)
        if child.cluster ==1:
            num_elems_secondclust+=1
            fcs_clust1.append(child.fc)
        intensities.append(child.min_intensity)

    if len(fcs_clust1)>0:
        betweenclust_fcfc = abs(np.mean(fcs_clust0) - np.mean(fcs_clust1))
    else:
        betweenclust_fcfc = 8

    stats_dict = {f"child_type_{node.type}_num_elems_secondclust" : num_elems_secondclust, f"child_type_{node.type}_num_clusters_total" : node.num_clusters, f"child_type_{node.type}_betweenclust_fcfc" : betweenclust_fcfc,
    f"child_type_{node.type}_num_elems_secondclust" : np.mean(intensities)}
    return stats_dict

def calc_variance_for_node(node):

    fcs_children = [x.fc for x in node.children]
    min_fc = min(fcs_children)
    fcs_children = [x.fc - min_fc for x in node.children]
    #print(f"fcs children are {fcs_children}, variance is {np.var(fcs_children)}")

    return scipy.stats.variation(fcs_children)

In [ ]:
#export
import anytree

def get_positive_negative_samples(condpairtree,type):
    all_nodes = []
    for protein_node in condpairtree.children:
        if protein_node.num_mainclusts <3:
            continue
        if protein_node.fraction_consistent < 0.7:
            continue
        type_nodes = anytree.search.findall(protein_node, filter_=lambda node: node.type == type)
        for type_node in type_nodes:
            fcdiff = abs(type_node.fc - protein_node.fc)

            if fcdiff<0.3:
                type_node.positive_example = True
                all_nodes.append(type_node)
                continue
            if fcdiff>2:
                type_node.positive_example = False
                all_nodes.append(type_node)
    return all_nodes

In [ ]:
#export
from sklearn.utils import resample
import alphaquant.classify_ions as aqclass

def get_tp_fp_nodes(condpairtree, nodetype, substantially_off_thresholds, close_to_target_threshold, precursor2fc = None):
    res_nodes =[]
    condpairtree.type = 'condpair'
    nodes = anytree.search.findall(condpairtree, filter_=lambda node:  (node.type == nodetype))
    if precursor2fc is not None:
        node2fc = get_node2fc(precursor2fc, nodes)

    for node in nodes:
        if precursor2fc is not None:
            fc = node2fc.get(node)
        else:
            fc = node.fc
        if (abs(fc) > substantially_off_thresholds[0]) & (abs(fc) < substantially_off_thresholds[1]):
            node.positive_example = False
            res_nodes.append(node)
        if abs(fc) < close_to_target_threshold:
            node.positive_example = True
            res_nodes.append(node)
    return res_nodes

In [ ]:
#export
import re
def get_node2fc(precursor2fc, nodes):
    node2fc = {}
    pattern = "(SEQ_.*_MOD_)(.*)(_CHARGE_)(.*)(_)"
    if nodes[0].name in precursor2fc.keys():
        rename_precursor= False
    else:
        rename_precursor = True
    for node in nodes:
        if rename_precursor:
            matched = re.search(pattern,node.name)
            precursor = f"{matched.group(2)}.{matched.group(4)}"
        else:
            precursor = node.name
        fc = precursor2fc.get(precursor)
        node2fc[node] = fc

    return node2fc

In [ ]:
#export
import alphaquant.visualizations as aqplot
def generate_ml_input(c1, c2, neg_thresholds =[1.5, 3], pos_threshold =  0.2, features_to_exclude = [], precursor2fc = None, results_folder="results_renamed"):
    tree = aqplot.read_condpair_tree(c1, c2, results_folder)
    pos_negs = get_tp_fp_nodes(tree, 'mod_seq_charge', neg_thresholds, pos_threshold, precursor2fc)
    df_precursor_features = aqclass.collect_node_parameters(pos_negs)
    df_precursor_features = df_precursor_features.drop(columns = features_to_exclude)

    df_precursor_features = balance_classes(df_precursor_features)
    #df_precursor_features = df_precursor_features.dropna()
    ionnames = df_precursor_features["ion"]
    df_precursor_features = df_precursor_features.drop(columns = "ion")
    df_precursor_features = df_precursor_features.astype('float')
    df_precursor_features = df_precursor_features.replace(np.nan, -1)

    X_outliers = df_precursor_features.drop(columns=["positive_example"]).to_numpy()
    print("shapes:")
    print(X_outliers.shape)
    y_outliers = df_precursor_features["positive_example"]
    print(y_outliers.shape)
    featurenames = list(df_precursor_features.drop(columns=["positive_example"]).columns)
    return X_outliers, y_outliers, featurenames, ionnames

In [ ]:
#export
import anytree
def ml_filter_and_plot_fcs(ml_classifier, threshold_for_negative_classification, c1, c2, results_folder="results_renamed"):
    results_df = aqplot.get_diffresult_dataframe(c1, c2, results_folder=results_folder)
    tree = aqplot.read_condpair_tree(c1, c2, results_folder)
    tree.type = "condpair"
    nodes = anytree.findall(tree, filter_= lambda x : x.type == 'mod_seq_charge')
    ion2isincluded = get_ion2classification(ml_classifier, nodes, threshold_for_negative_classification)
    results_df = results_df[[ion2isincluded.get(x) for x in results_df["protein"]]]

    aqplot.volcano_plot(results_df)

In [ ]:
#export
def get_ion2classification(ml_classifier, nodes, threshold_for_positive_classification):
    df_precursor_features = aqclass.collect_node_parameters(nodes, w_annot=False)
    df_precursor_features = df_precursor_features.astype('float')
    df_precursor_features = df_precursor_features.replace(np.nan, -1)
    X = df_precursor_features.to_numpy()
    ionnames = [x.name for x in nodes]
    probabilities_positive_example = [x[1] for x in ml_classifier.predict_proba(X)]
    ion2included = {ion:(prob>threshold_for_positive_classification ) for ion, prob in zip(ionnames, probabilities_positive_example)}
    return ion2included

In [ ]:
#export

def balance_classes(df_precursor_features):
    df_pos_examples = df_precursor_features[df_precursor_features["positive_example"] ==1]
    df_neg_examples = df_precursor_features[df_precursor_features["positive_example"] ==0]
    min_length = min(len(df_pos_examples.index), len(df_neg_examples.index))
    df_pos_examples = resample(df_pos_examples, replace = False, n_samples = int(min_length), random_state = 123)
    df_neg_examples = resample(df_neg_examples, replace = False, n_samples = min_length, random_state = 123)
    df_downsampled = pd.concat([df_pos_examples, df_neg_examples], ignore_index=True)
    print(sum(df_downsampled["positive_example"]))
    print(len(df_downsampled.index))
    return df_downsampled

In [ ]:
#export
import numpy as np

from sklearn.utils import resample
def load_feature_df(c1, c2, neg_thresholds =[1.5, 3], pos_threshold =  0.2, features_to_exclude = []):
    tree = aqplot.read_condpair_tree(c1, c2, results_folder="results")
    pos_negs = get_tp_fp_nodes(tree, 'mod_seq_charge', neg_thresholds, pos_threshold)
    df_precursor_features = aqclass.collect_node_parameters(pos_negs)
    df_precursor_features = df_precursor_features.drop(columns = features_to_exclude)
    return df_precursor_features

In [ ]:
#export
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

def plot_precision_recall(classfier, X_train, y_train, X_test, y_test):
      y_score = classfier.decision_function(X_train)
      average_precision = average_precision_score(y_train, y_score)

      print('Average precision-recall score: {0:0.2f}'.format(
            average_precision))


      disp = plot_precision_recall_curve(classfier, X_test, y_test)
      disp.ax_.set_title('2-class Precision-Recall curve: '
                        'AP={0:0.2f}'.format(average_precision))
      plt.show()

In [ ]:
#export
def get_precursor2fc(cond1, cond2, results_folder):
    result_df = aqviz.get_diffresult_dataframe(cond1, cond2, results_folder)
    result_dict = dict(zip(result_df["protein"], result_df["log2fc"]))
    return result_dict

In [ ]:
#export
import alphaquant.diffquant_utils as aqutils

def get_nodes_of_type(cond1, cond2, results_folder, node_type = 'mod_seq_charge'):

    tree_sn = aqutils.read_condpair_tree(cond1, cond2, results_folder=results_folder)
    tree_sn.type = "asd"
    return anytree.findall(tree_sn, filter_= lambda x : (x.type == node_type))

In [ ]:
#export
import alphaquant.classify_ions as aqclass
import alphaquant.visualizations as aqplot
import anytree
import copy
import random
import numpy as np
def get_fc_normalized_nodes(nodes_fclevel, type_lowerlevel, min_nums_lowerlevel = 2, fc_cutoff = 1.0, distort_precursor_modulo = np.inf):
    """"get nodes of type lowerlevel which are normalized by the fclevel fold change"""
    randnr_generator = random.Random(42)
    normalized_lowerlevels = [] #the normalized lowerlevels are copied values used for training, better change to different variable names to be used
    all_lowerlevels = []
    count_precursors = 0
    for prot in nodes_fclevel:
        precursors = anytree.findall(prot, filter_= lambda x : (x.type == type_lowerlevel))
        all_lowerlevels.extend(precursors)
        if len(precursors)<min_nums_lowerlevel:
            continue
        fc_prot = prot.fc
        if abs(fc_prot) < fc_cutoff:
            continue
        for precursor in precursors: #shift every peptide by the protein fold change, this should make peptides from different proteins comparable and enable analyses/ml on this dataset
            precursor = copy.copy(precursor)
            precursor.fc = precursor.fc - fc_prot
            if count_precursors%distort_precursor_modulo==0:
                perturbation = randnr_generator.uniform(-2, 2)
                precursor.fc=precursor.fc + perturbation
                precursor.perturbation_added = perturbation
            normalized_lowerlevels.append(precursor)
            count_precursors+=1

    return normalized_lowerlevels, all_lowerlevels

In [ ]:
#export

def get_intersect_sn_diann_precursors(cond1, cond2, sn_folder, diann_folder):
    precursor_nodes_diann = get_nodes_of_type(cond1, cond2, diann_folder, node_type = 'mod_seq_charge')
    precursor_nodes_sn = get_nodes_of_type(cond1, cond2, sn_folder, node_type = 'mod_seq_charge')
    precursors_diann = {x.name.replace("_MOD_", "_MOD__").replace("_CHARGE", "__CHARGE") for x in precursor_nodes_diann}
    precursors_sn = {x.name for x in precursor_nodes_sn}

    intersect = precursors_diann.intersection(precursors_sn)

    return intersect

In [ ]:
#export
def replace_nans_feature_dependent(df_precursor_features):
    colums_lower_is_worse = [x for x in  df_precursor_features.columns if 'frac_mainclust' in x]
    columns_higher_is_worse = [x for x in df_precursor_features.columns if x not in colums_lower_is_worse]
    df_precursor_features[colums_lower_is_worse] = df_precursor_features[colums_lower_is_worse].replace(np.nan, 0)
    df_precursor_features[columns_higher_is_worse] = df_precursor_features[columns_higher_is_worse].replace(np.nan, 10)

    return df_precursor_features


In [ ]:
#export

def generate_ml_input_regression(df_precursor_features, nodes, replace_nans = False):
    ion2fc = {x.name: x.fc for x in nodes}
    if replace_nans:
        df_precursor_features = replace_nans_feature_dependent(df_precursor_features)
    else:
        df_precursor_features = df_precursor_features.dropna()
    df_precursor_features = df_precursor_features[[(x in ion2fc.keys()) for x in df_precursor_features["ion"]]]
    ionnames = list(df_precursor_features["ion"])
    df_precursor_features = df_precursor_features.drop(columns=["ion"])
    X = df_precursor_features.to_numpy()
    y = np.array([ion2fc.get(ion) for ion in ionnames])
    featurenames = list(df_precursor_features.columns)
    return X, y, featurenames, ionnames

In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np
import alphaquant.visualizations as aqplot

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import alphaquant.classify_ions as aqclass
from sklearn import datasets, linear_model


def do_linear_regression(X, y, featurenames, ionnames, prediction_cutoff):
    regr = linear_model.LinearRegression()
    X = scale_input(X)
    # Split the data into training/testing sets
    fc2ion = {x:y for x,y in zip(y, ionnames)}
    X_train, X_test, y_train, y_test = train_test_split(X, y)


    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print('Mean squared error: %.2f'
        % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
        % r2_score(y_test, y_pred))

    # Plot outputs
    #plt.plot(X_test[:,-1], y_pred,  color='black')
    plt.scatter(y_test, y_pred, color='blue')
    plt.show()
    aqclass.plot_feature_importances(regr.coef_,featurenames, 10)
    plt.show()
    aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=False)
    plt.show()
    aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=True)
    plt.show()
    print_good_predicitions(y_test, y_pred, fc2ion)

In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor



def do_random_forest_regression(X, y, featurenames, prediction_cutoff):
      # Split the data into training/testing sets
      X = scale_input(X)
      X_train, X_test, y_train, y_test = train_test_split(X, y)

      # Create linear regression object
      regr = RandomForestRegressor(max_depth=2, random_state=0)

      # Train the model using the training sets
      regr.fit(X_train, y_train)

      # Make predictions using the testing set
      y_pred = regr.predict(X_test)


      # The mean squared error
      print('Mean squared error: %.2f'
            % mean_squared_error(y_test, y_pred))
      # The coefficient of determination: 1 is perfect prediction
      print('Coefficient of determination: %.2f'
            % r2_score(y_test, y_pred))

      # Plot outputs
      #plt.plot(X_test[:,-1], y_pred,  color='black')
      plt.scatter(y_test, y_pred, color='blue')
      plt.show()
      aqclass.plot_feature_importances(regr.feature_importances_,featurenames, 10)
      plt.show()
      aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=False)
      plt.show()
      aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=True)
      plt.show()

In [ ]:
#export

import sklearn.preprocessing

def scale_input(X):
    scaler = sklearn.preprocessing.StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    return X_scaled

def scale_input_minmax(X):
    scaler = sklearn.preprocessing.MinMaxScaler()
    X_transformed = scaler.transform(X)
    return X_transformed


def print_good_predicitions(y_test, y_pred, fc2ionnames,top_n = 10):
    tuples = list(zip(y_test, y_pred))
    tuples.sort(key = lambda x: min(abs(np.array([x[0], x[1]]))), reverse=True)
    print('printing names of some well predicted ions')
    for idx in range(top_n):
        print(f"{tuples[idx]}\t{fc2ionnames.get(tuples[idx][0])}")

In [ ]:
#export
import random
import math

def get_indices_for_cross_predict(y, number_splits):
    length_subset = math.floor(len(y)/number_splits)
    y_idxs = list(range(len(y)))
    random.shuffle(y_idxs)
    chunks_excluded = []
    chunks_included = []
    for i in range(0, len(y), length_subset):
        idxs_excluded = y_idxs[i:i + length_subset]
        idxs_included = [idx for idx in y_idxs if idx not in idxs_excluded]
        chunks_excluded.append(idxs_excluded)
        chunks_included.append(idxs_included)
    return chunks_included, chunks_excluded

In [ ]:
#export

def get_trained_predictor_and_predicted_trainset(X, y, ionnames, number_splits, regr):
    y = np.array(y)
    X = scale_input(X)

    chunks_included, chunks_excluded = get_indices_for_cross_predict(y, number_splits)
    y_test_all = []
    y_pred_all = []
    ionnames_all = []
    all_excluded = {}
    for i in range(len(chunks_included)):
        idxs_in = chunks_included[i]
        idxs_out = chunks_excluded[i]

        if len(set(idxs_out).intersection(all_excluded))>0:
            raise Exception('train set in test set!')

        X_train = X[idxs_in,:]
        y_train = y[idxs_in]
        X_test = X[idxs_out, :]
        y_test = y[idxs_out]
        regr.fit(X_train, y_train)
        # Make predictions using the testing set
        y_pred = regr.predict(X_test)
        print(f"round {i} w. {len(idxs_in)} ions used for training")
        y_test_all.extend(y_test)
        y_pred_all.extend(y_pred)
        ionnames_all.extend(ionnames[idxs_out])

    return y_test_all, y_pred_all, ionnames_all, regr

In [ ]:
#export
import alphaquant.diffquant_utils as aqutils

def predict_remaining_dataset(all_precursors, regr, y_pred_cp, ionnames_cp, acquisition_info_df):
    precursors_to_predict = [x for x in all_precursors if x.name not in ionnames_cp]
    df_precursor_features = collect_node_parameters(precursors_to_predict)
    merged_df = aqutils.merge_acquisition_df_parameter_df(acquisition_info_df, df_precursor_features)
    X_remaining  = generate_ml_input_regression(merged_df, precursors_to_predict, replace_nans=True)[0]
    y_pred_remaining = regr.predict(X_remaining)
    ionnames_remaining = [x.name for x in precursors_to_predict]
    y_pred_total = list(y_pred_cp) + list(y_pred_remaining)
    ionnames_total = list(ionnames_cp) + list(ionnames_remaining)

    return y_pred_total, ionnames_total

In [ ]:
#export
import scipy.stats
from sklearn.mixture import GaussianMixture
import alphaquant.visualizations as aqplot
import alphaquant.normalization as aqnorm

def find_nearest(array, values):
    indices = np.abs(np.subtract.outer(array, values)).argmin(0)
    return indices

def find_mean_and_cutoffs(y_pred, visualize = False):
    data = np.array(y_pred)
    gmm = GaussianMixture(n_components = 2).fit(data.reshape(-1, 1))
    if visualize:
        aqplot.visualize_gaussian_mixture_fit(gmm,y_pred)
        plt.hist(data, bins=50, histtype='step', density=True, alpha=0.5, label='before shift')
    weight_mean_cov = list(zip(gmm.weights_, gmm.means_, gmm.covariances_))
    weight_mean_cov.sort(key = lambda x : x[0], reverse=True)
    weight_mean_cov = weight_mean_cov[0] # get the ND with the largest weight
    mean = weight_mean_cov[1][0]
    stdev = np.sqrt(weight_mean_cov[2][0][0])
    val_neg = scipy.stats.norm.ppf(0.01,  scale = stdev)
    val_pos = scipy.stats.norm.ppf(0.99,  scale = stdev)
    print(f"the mean is {mean}")
    if visualize:
        plt.hist(y_pred-mean, bins=50, histtype='step', density=True, alpha=0.5, label= 'after_shift')
        plt.legend()
        plt.show()
    return mean ,val_neg, val_pos


def fit_gaussian_to_dist_around_mode(dist, visualize = False):
    #cut out the "core" ~50% of the distribution, which I assume to be gaussian, then fit a gaussian to this core and determine mean and stdev
    dist = np.sort(np.array(dist))
    mode = aqnorm.determine_mode_iteratively(dist)
    mode_idx = find_nearest(dist, mode)

    cumsum_rel = np.array(list(range(len(dist))))/len(dist)
    mode_quantile = cumsum_rel[mode_idx]
    subrange = min([mode_quantile, 0.34, 1-mode_quantile]) #if the mode is close to the edge, make the interval smaller

    print(f"subrange: {subrange}")
    quantile1 = mode_quantile-subrange
    quantile2 = mode_quantile+subrange
    idx_start = find_nearest(cumsum_rel, quantile1)
    idx_end = find_nearest(cumsum_rel, quantile2)
    print(f'before_adjust: start {idx_start} mode {mode_idx} end {idx_end}')
    #try to fit around the same "widht" around the mode
    difference_start_mode = mode_idx - idx_start
    difference_mode_end = idx_end - mode_idx
    if difference_start_mode>difference_mode_end:
        idx_end = mode_idx+difference_start_mode
    if difference_mode_end> difference_start_mode:
        idx_start = mode_idx-difference_mode_end
    print(f'after adjust: start {idx_start} mode {mode_idx} end {idx_end}')

    dist_subset = dist[idx_start:idx_end]
    gmm = GaussianMixture(n_components = 1).fit(dist_subset.reshape(-1, 1))
    mean = gmm.means_[0]
    var = gmm.covariances_[0]
    empirical_stdev = (dist[idx_end] - dist[idx_start])/2
    if visualize:
        aqplot.visualize_gaussian_nomix_subfit(mode, (empirical_stdev)**2,dist,dist_subset)
        plt.hist(dist, bins=50, histtype='step', density=True, alpha=0.5, label='before shift')
        plt.hist(dist_subset,  bins=25, histtype='step', density=True, alpha=0.5, label='before shift')
        plt.show()
    mean = gmm.means_[0]
    stdev = np.sqrt(gmm.covariances_[0])
    val_neg = scipy.stats.norm.ppf(1e-5,  scale = empirical_stdev)
    val_pos = scipy.stats.norm.ppf(1-1e-5,  scale = empirical_stdev)
    return mode, val_neg, val_pos


def gauss(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))


def gaussian(x, amp, cen, wid):
    """1-d gaussian: gaussian(x, amp, cen, wid)"""
    return (amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2))


from lmfit import Model
from scipy.stats import norm
def fit_gaussian_to_subdist(dist, visualize, results_dir = None):
    #cut out the middle of the data
    dist = np.sort(np.array(dist))
    dist_hist = np.histogram(dist, bins = int(len(dist)/3))
    counts= dist_hist[0]
    bins = dist_hist[1]
    bins_middled = [(bins[i]+bins[i+1])/2 for i in range(len(bins)-1)]
    
    mode = aqnorm.determine_mode_iteratively(dist)
    mode_idx = find_nearest(bins_middled, mode)

    cumsum_rel = np.cumsum(counts)/sum(counts)

    mode_quantile = cumsum_rel[mode_idx]
    subrange = min([mode_quantile, 0.4, 1-mode_quantile]) #if the mode is close to the edge, make the interval smaller

    print(f"subrange: {subrange}")
    quantile1 = mode_quantile-subrange
    quantile2 = mode_quantile+subrange
    idx_start = 0#find_nearest(cumsum_rel, quantile1)
    idx_end = len(cumsum_rel)-1#find_nearest(cumsum_rel, quantile2)
    x = bins_middled[idx_start:idx_end]
    y = counts[idx_start:idx_end]


    gmodel = Model(gaussian)
    result = gmodel.fit(y, x=x, amp=5, cen=0, wid=1)
    fit_results = result.__dict__["best_values"]
    mean = fit_results["cen"]
    stdev = fit_results["wid"]
    weight = fit_results["amp"]

    if visualize:
        plt.hist(dist, bins=int(len(dist)/3), alpha = 0.5)
        fit_results = weight*norm.pdf(bins_middled, mean, stdev).ravel()
        plt.plot(bins_middled, fit_results, '-', label = 'best fit',linewidth = 1.5, c = 'black')
        #plt.plot(x, result.best_fit, '-', label='best fit')
        plt.legend()
        if results_dir is not None:
            plt.savefig(f'{results_dir}/ml_offsets_gaussian_fit.pdf')
        plt.show()
    
    print(stdev)
    #val_neg = scipy.stats.norm.ppf(0.001,  scale = stdev)
    #val_pos = scipy.stats.norm.ppf(0.999,  scale = stdev)
    val_neg = -3*stdev
    val_pos = 3*stdev
    print(mean, val_neg, val_pos)
    return mean, val_neg, val_pos
    

In [ ]:
#export
def annotate_precursor_nodes(cutoff_neg, cutoff_pos, y_pred_total, ionnames_total, precursor_nodes):
    precursor2predscore = {x:y for x, y in zip(ionnames_total, y_pred_total)}
    for precursor in precursor_nodes:
        predscore = precursor2predscore.get(precursor.name)
        ml_excluded = not ((predscore>cutoff_neg) & (predscore<cutoff_pos))
        precursor.predscore = predscore
        precursor.ml_excluded = bool(ml_excluded)
        precursor.cutoff = cutoff_pos #cutoff_pos is -cutoff_neg


In [ ]:
#export
import alphaquant.visualizations as aqplot
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import random
import numpy as np


def balance_small_and_strong_fcs(normed_nodes, cutoff):
    smaller_pep_idxs = []
    larger_peps_idxs = []
    for idx in range(len(normed_nodes)):
        fc = normed_nodes[idx].fc
        if abs(fc)< cutoff:
            smaller_pep_idxs.append(idx)
        else:
            larger_peps_idxs.append(idx)
    if len(larger_peps_idxs)< len(smaller_pep_idxs):
        print("balancing subsets")
        print(f"{len(larger_peps_idxs)} of large vs {len(smaller_pep_idxs)} of small fcs")
        smaller_pep_idxs = random.sample(smaller_pep_idxs, len(larger_peps_idxs))
    else:
        print("skip balancing subsets")
        return normed_nodes
    new_idxs = smaller_pep_idxs+larger_peps_idxs
    return [normed_nodes[i] for i in range(len(new_idxs))]


def random_forest_iterative_cross_predict(X, y, ionnames, number_splits, regr, balancing_cutoff = np.inf):
    X = scale_input(X)
    ionnames = np.array(ionnames)
    #X_balanced, y_balanced, ionnames_balanced = balance_small_and_strong_fcs(X, y, ionnames,balancing_cutoff)
    #X_not_balanced, y_not_balanced  = get_balance_excluded_subset(y_balanced, y, X)

    chunks_included, chunks_excluded = get_indices_for_cross_predict(y, number_splits)
    y_test_all = []
    y_pred_all = []
    ionnames_all = []
    all_excluded = {}
    for i in range(len(chunks_included)):
        idxs_in = chunks_included[i]
        idxs_out = chunks_excluded[i]

        if len(set(idxs_out).intersection(all_excluded))>0:
            print("overfitting alarm!")
            all_excluded.update(set(idxs_out))

        X_train = X[idxs_in,:]
        y_train = y[idxs_in]
        X_test = X[idxs_out, :]
        y_test = y[idxs_out]
        regr.fit(X_train, y_train)
        if i==0:
            regr_export = regr
        # Make predictions using the testing set
        y_pred = regr.predict(X_test)
        
        y_test_all.extend(y_test)
        y_pred_all.extend(y_pred)
        ionnames_all.extend(ionnames[idxs_out])

    return y_test_all, y_pred_all, ionnames_all, regr_export
    #aqplot.get_error_and_scatter_ml_regression(y_test_all, y_pred_all, scatter_filt)



import random
import math

def get_indices_for_cross_predict(y, number_splits):
    length_subset = math.floor(len(y)/number_splits)
    y_idxs = list(range(len(y)))
    random.Random(42).shuffle(y_idxs) #set seed 42
    chunks_excluded = []
    chunks_included = []
    for i in range(0, len(y), length_subset):
        idxs_excluded = y_idxs[i:i + length_subset]
        idxs_included = [idx for idx in y_idxs if idx not in idxs_excluded]
        chunks_excluded.append(idxs_excluded)
        chunks_included.append(idxs_included)
    return chunks_included, chunks_excluded


def get_balance_excluded_subset(y_balanced, y, X):
    idxs_y_balance_excluded = [x for x in range(len(y)) if (y[x] not in y_balanced)]
    y_not_balanced = np.array(y)[idxs_y_balance_excluded]
    X_not_balanced = X[idxs_y_balance_excluded,:]
    return X_not_balanced, y_not_balanced


def calculate_log_loss_scores_for_prediction(y_test, y_pred):
    loss_scores = abs(np.log2(y_test) - np.log2(y_pred))
    return loss_scores

In [ ]:
import numpy.random
import copy
def test_iterative_cross_predict():
    X = numpy.random.rand( 200, 3)
    y = numpy.random.rand(200)
    ionnames = numpy.random.rand(200)
    select_idxs = numpy.random.randint(low = 0, high = 199, size = 15)
    control_ionnames = copy.copy(ionnames[select_idxs])
    

    y_test_all, _, ionnames_all, _ = random_forest_iterative_cross_predict(X, y, ionnames, 5, RandomForestRegressor())

    idxs_ionnames = [x for x in range(len(ionnames_all)) if ionnames_all[x] in control_ionnames]
    control_idxs_y = [x for x in select_idxs if ionnames[x] in ionnames_all]
    control_ys = y[control_idxs_y]

    y_test_all_control = np.array(y_test_all)[idxs_ionnames]

    assert set(control_ys).intersection(set(y_test_all_control)) == set(y_test_all_control)


test_iterative_cross_predict()


[24, 21, 87, 5, 49, 13, 121, 129, 141, 143, 92, 157, 23, 97, 45, 108, 27, 63, 55, 173]
[176, 4, 103, 61, 111, 184, 163, 181, 75, 131, 78, 107, 198, 123, 47, 35, 1, 91, 33, 195]
[51, 62, 118, 167, 31, 72, 151, 25, 155, 8, 89, 65, 169, 179, 36, 178, 137, 194, 159, 12]
[88, 140, 191, 142, 18, 160, 76, 125, 161, 41, 73, 77, 58, 93, 3, 114, 166, 113, 124, 15]
[149, 37, 133, 7, 71, 146, 139, 53, 44, 164, 126, 60, 119, 70, 153, 68, 85, 57, 115, 152]


In [ ]:
#export
import alphaquant.diffquant_utils as aqutils
import alphaquant.visualizations as aqplot
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

def assign_predictability_scores(protein_nodes, results_dir, name, samples_used,precursor_cutoff=2, fc_cutoff = 1.0, number_splits = 5, plot_predictor_performance = False, replace_nans = False, distort_precursor_modulo = np.inf, performance_metrics = {}):

    #add predictability scores to each precursor
    #prepare the input table with all the relevant features for machine learning
    dfhandler = aqutils.AcquistionDataFrameHandler(results_dir=results_dir,samples=samples_used)
    acquisition_info_df = get_acquisition_info_df(dfhandler)
    node_level = get_node_level_from_dfhandler(dfhandler)
    protein_nodes = list(sorted(protein_nodes, key  = lambda x : x.name))
    normalized_precursors, all_precursors = get_fc_normalized_nodes(protein_nodes, node_level, precursor_cutoff, fc_cutoff, distort_precursor_modulo=distort_precursor_modulo)
    df_precursor_features = collect_node_parameters(normalized_precursors)
    merged_df = aqutils.merge_acquisition_df_parameter_df(acquisition_info_df, df_precursor_features)
    
    #transform into ML input
    X, y, featurenames, ionnames = generate_ml_input_regression(merged_df, normalized_precursors, replace_nans=replace_nans)

    test_fc_name_mapping(y, ionnames, normalized_precursors)

    #predict the subset of peptides that is accessible to protein shifting (only the ones with at least 2 peps per protein)
    regr = RandomForestRegressor()
    y_test_cp, y_pred_cp, ionnames_cp, regr = random_forest_iterative_cross_predict(X, y, ionnames, number_splits, regr)
    print("performed RF prediction")
    performance_metrics["r2_score"] = r2_score(y_test_cp, y_pred_cp)
    test_fc_name_mapping(y_test_cp, ionnames_cp, normalized_precursors)

    #define plot outdir
    results_dir_plots =f"{results_dir}/{name}" 
    aqutils.make_dir_w_existcheck(results_dir_plots)
    if plot_predictor_performance:
        plt.hist(y_test_cp, 60, density=True, histtype='stepfilled',cumulative=False, alpha = 0.5)
        plt.xlim(-1.8, 1.8)
        plt.show()
        
        aqplot.scatter_ml_regression_perturbation_aware(y_test=y_test_cp, y_pred = y_pred_cp, ionnames=ionnames_cp, nodes=normalized_precursors, results_dir=results_dir_plots)
        aqplot.plot_ml_fc_histograms(y_test_cp, y_pred_cp, 0.5, results_dir_plots)
        aqplot.plot_feature_importances(regr.feature_importances_,featurenames, 10, results_dir_plots)
    
    #mean, cutoff_neg, cutoff_pos = find_mean_and_cutoffs(y_pred_cp, visualize= plot_predictor_performance)
    mean, cutoff_neg, cutoff_pos = fit_gaussian_to_subdist(y_pred_cp, visualize=plot_predictor_performance,results_dir = results_dir_plots)
    #use the trained model to predict the remaining ions and stitch together with the already predicted ions
    y_pred_total, ionnames_total = predict_remaining_dataset(all_precursors, regr, y_pred_cp, ionnames_cp, acquisition_info_df)

    y_pred_normed = y_pred_total-mean

    #add quality scores to nodes

    add_quality_scores_to_node(acquisition_info_df, all_precursors)

    #annotate the precursor nodes
    annotate_precursor_nodes(cutoff_neg, cutoff_pos, y_pred_normed, ionnames_total, all_precursors) #two new variables added to each node:


def get_acquisition_info_df(dfhandler):
    acquisition_df = dfhandler.get_acquisition_info_df()
    if not dfhandler.already_formatted:
        dfhandler.save_allsample_dataframe_as_new_acquisition_dataframe()
        dfhandler.update_ml_file_location_in_method_parameters_yaml()
    return acquisition_df

from alphaquant.cluster_ions import globally_initialized_typefilter
def get_node_level_from_dfhandler(dfhandler):
    return globally_initialized_typefilter.mapping_dict.get(dfhandler.last_ion_level_to_use)


def add_quality_scores_to_node(acquisition_info_df, nodes):
    #check if the data belongs to Spectronaut or DIANN
    if "FG.ShapeQualityScore" in acquisition_info_df.columns:
        param = "FG.ShapeQualityScore"
    elif "Quantity.Quality" in acquisition_info_df.columns:
        param = "Quantity.Quality"
    else:
        return
    
    df_avged = acquisition_info_df.groupby("ion").mean().reset_index()
    ion2param = dict(zip(df_avged["ion"], df_avged[param]))
    
    for node in nodes:
        node.default_quality_score = ion2param.get(node.name)

def test_fc_name_mapping(fcs, ionnames, nodes):
    number_samplings = min(math.ceil(len(fcs)/5), 30)
    sample_idxs = random.sample(range(len(fcs)), number_samplings)
    name2node = {x.name : x for x in nodes}
    for idx in sample_idxs:
        fc_ml_input = fcs[idx]
        name_ml_input = ionnames[idx]
        fc_node = name2node.get(name_ml_input).fc
        assert fc_ml_input == fc_node



